In [25]:

import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer



In [26]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure nltk data is downloaded
nltk.data.path.append(r"C:\Users\ASHISHI RANJAN\AppData\Roaming\nltk_data")
nltk.download('punkt', force=True)
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    tokens = word_tokenize(text)  # Tokenize
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]  # Remove stopwords
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize
    return ' '.join(tokens)

[nltk_data] Downloading package punkt to C:\Users\ASHISHI
[nltk_data]     RANJAN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to C:\Users\ASHISHI
[nltk_data]     RANJAN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\ASHISHI
[nltk_data]     RANJAN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [27]:
print(nltk.data.find('tokenizers/punkt'))


C:\Users\ASHISHI RANJAN\AppData\Roaming\nltk_data\tokenizers\punkt


In [28]:
# def preprocess_text(text):
#     # Tokenize text
#     tokens = word_tokenize(text)

#     # Remove stop words
#     stop_words = set(stopwords.words('english'))
#     tokens = [token for token in tokens if token.lower() not in stop_words]

#     # Lemmatize tokens
#     lemmatizer = WordNetLemmatizer()
#     tokens = [lemmatizer.lemmatize(token) for token in tokens]

#     return ' '.join(tokens)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

def text_representation(text):
    vectorizer = TfidfVectorizer()
    vectorizer.fit([preprocess_text(text)])  # Fit TF-IDF
    vectors = vectorizer.transform([preprocess_text(text)])  # Transform text to vector
    return vectors

In [30]:
# from sumy.parsers.plaintext import PlaintextParser
# from sumy.nlp.tokenizers import Tokenizer
# from sumy.summarizers.lsa import LsaSummarizer

# def summarize_text(text):
#     parser = PlaintextParser.from_string(text, Tokenizer("english"))
#     summarizer = LsaSummarizer()
#     summary = summarizer(parser.document, 3)  # Number of sentences
#     return " ".join(str(sentence) for sentence in summary)


In [31]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_text(text, min_words=50, max_words=200):
    # Generate summary with BART while ensuring word constraints
    raw_summary = summarizer(text, max_length=max_words, min_length=min_words, do_sample=False)[0]['summary_text']

    # Trim summary to exact word range
    words = raw_summary.split()
    num_words = min(len(words), max(1, (min_words + max_words) // 2))  # Balanced output
    final_summary = " ".join(words[:num_words])

    return final_summary


All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.
Device set to use 0


In [32]:
def post_process_summary(summary):
    # Remove extra spaces
    summary = ' '.join(summary.split())

    return summary

In [33]:
text = """NLTK (Natural Language Toolkit) is a popular Python library for natural language processing (NLP). It provides us various text processing libraries with a lot of test datasets. A variety of tasks can be performed using NLTK such as tokenizing, parse tree visualization, etc… In this article, we will go through how we can set up NLTK in our system and use them for performing various NLP tasks during the text processing step. Tokenization:
The breaking down of text into smaller units is called tokens. tokens are a small part of that text. If we have a sentence, the idea is to separate each word and build a vocabulary such that we can represent all words uniquely in a list. Numbers, words, etc.. all fall under tokens.

Lower case conversion:
We want our model to not get confused by seeing the same word with different cases like one starting with capital and one without and interpret both differently. So we convert all words into the lower case to avoid redundancy in the token list. Stop Words removal:
When we use the features from a text to model, we will encounter a lot of noise. These are the stop words like the, he, her, etc… which don’t help us and, just be removed before processing for cleaner processing inside the model. With NLTK we can see all the stop words available in the English language.

from nltk.corpus import stopwords"""
summary = summarize_text(text)
print("Final Summary:", post_process_summary(summary))

Final Summary: NLTK (Natural Language Toolkit) is a popular Python library for natural language processing (NLP) It provides us various text processing libraries with a lot of test datasets. A variety of tasks can be performed using NLTK such as tokenizing, parse tree visualization, etc…


In [37]:
import openai
import sentencepiece
from transformers import T5Tokenizer

t5_model_name = "t5-large"
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_name)
print("T5Tokenizer loaded successfully!")


ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:
import gradio as gr
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline
import openai
import sentencepiece
import torch

# Load T5 for long summary generation
t5_model_name = "t5-large"
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_name)

# Load BART for standard summarization
bart_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# OpenAI GPT-4 API key (Make sure to set up your key in an environment variable)
OPENAI_API_KEY = "your_openai_api_key_here"
openai.api_key = OPENAI_API_KEY


### 1️⃣ Function: Summarize a Long Text (Re-Summarization) ###
def summarize_text_t5(text, min_words=1000, max_words=2000):
    """Generate a long summary directly from the original text using T5."""
    prompt = f"summarize: {text} </s>"
    inputs = t5_tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

    summary_ids = t5_model.generate(
        **inputs, 
        max_length=max_words, 
        min_length=min_words, 
        length_penalty=2.0, 
        num_beams=4
    )

    summary = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


### 2️⃣ Function: Expand a Short Summary to Longer One ###
def expand_summary_gpt4(summary, target_word_count=2000):
    """Expand an existing summary to a longer one using GPT-4."""
    prompt = f"Expand the following summary to {target_word_count} words by adding details, examples, and explanations:\n\n{summary}"

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "You are an expert at expanding summaries."},
                  {"role": "user", "content": prompt}]
    )

    return response["choices"][0]["message"]["content"]


### Gradio UI ###
def summarizer_interface(text, min_words, max_words, expand_summary=False):
    """Handles both summarization and expansion based on user selection."""
    
    # First, generate an initial summary
    initial_summary = summarize_text_t5(text, min_words, max_words)
    
    # If the user wants to expand it further, pass it to GPT-4
    if expand_summary:
        final_summary = expand_summary_gpt4(initial_summary, max_words)
        return final_summary
    else:
        return initial_summary


iface = gr.Interface(
    fn=summarizer_interface,
    inputs=[
        gr.Textbox(label="Enter Text to Summarize"),
        gr.Slider(500, 5000, step=100, label="Min Summary Length (words)"),
        gr.Slider(500, 5000, step=100, label="Max Summary Length (words)"),
        gr.Checkbox(label="Expand Summary with GPT-4")
    ],
    outputs="text",
    title="Advanced Summarizer (Re-Summarization & Expansion)"
)

iface.launch(share=True)


ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
